In [14]:
###pip is the package installer for python3. The command "py -m pip install"
### installs the packages where in "-m pip" executes pip using the latest python
### interpreter installed in windows.

import os
import subprocess
import re

packagelist = "openpyxl", "beautifulsoup4"

for package in packagelist:
    try:
        import package
    except:
        subprocess.call(["py","-m","pip", "install", package], stdout=subprocess.DEVNULL)
       # os.system("py -m pip install " + package)

from openpyxl import Workbook
from openpyxl import load_workbook
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains


In [35]:
###########Input
#print("BRCA1/BRCA2?")
#brca_input = input()
brca_input = "BRCA1"
brca_input = brca_input.upper()
#print("Search?")
#search = input()
search = "4308"

In [36]:


####Setting up the location of the excel files

folder_location = "C:/Users/jensv/Desktop/BRCA_-_prostate_cancer_and_ovarian_cancer_-_PARPi"
functional_categorization = "/2020 BRCA1 functional categorization of BRCA1 VUS - CCR.xlsx"
variants_excel = "/allenigmavariants_BICsubmission_2013-07-01.xlsx"
bicbnl = "/bicbnl 27  fh 2-10-13_nov2013.xlsx"
enigma = "/BRCA_Multifac_published data_ENIGMAwebsite 2015-03-27.xlsx"

In [6]:
##################Write data to excel
#Writing data to word
from docx import Document
from docx.shared import Cm
#py -m pip install --user python-docx
document = Document()

document.add_heading("2020 functional categorization of BRCA1 VUS - CCR", 1)

table = document.add_table(rows=1, cols=4)

table.style = 'Colorful List Accent 1'
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'DNA variant'
hdr_cells[1].text = 'Cisplatin Assay'
hdr_cells[2].text = 'Olaparib Assay'
hdr_cells[3].text = 'DR-GFP assay'

document.save('test.docx')


In [52]:
#1) Reading excel sheet: 2020 functional categorization of BRCA1 VUS - CCR
#The script will load the excel workbook and sheet and create empty lists of the different assays and found dna variants
#it will iterate over each cell and append the results to the lists if the input is found in the first column: DNA variant

##################Write data to excel
#Writing data to word
from docx import Document
from docx.shared import Cm
#py -m pip install --user python-docx
document = Document()

document.add_heading("2020 functional categorization of BRCA1 VUS - CCR", 1)

table = document.add_table(rows=1, cols=4)

table.style = 'Colorful List Accent 1'
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'DNA variant'
hdr_cells[1].text = 'Cisplatin Assay'
hdr_cells[2].text = 'Olaparib Assay'
hdr_cells[3].text = 'DR-GFP assay'


if brca_input == "BRCA1":

    wb = load_workbook(folder_location + functional_categorization)

    sheet = wb.active

    for row in sheet.iter_rows(max_col = 1):
        for cell in row:
            if search in str(cell.value):
                row_cells = table.add_row().cells
                Result = True
                if Result == True:
                    row_cells[0].text = cell.value
                    row_cells[1].text = cell.offset(column=2).value
                    row_cells[2].text = cell.offset(column=3).value
                    row_cells[3].text = cell.offset(column=4).value
                    Result = False

document.save('test.docx')

NameError: name 'folder_location' is not defined

In [51]:
#2) Reading excel sheet : allenigmavariants_BICsubmission_2013-07-01
#The script will iterate over cell but will only append to the lists if the search is found in the cell and the right BRCA input is present in the first column
#Another elif statement is added which seperates the search found between HGVS and BIC

wb = load_workbook(folder_location + variants_excel)

sheet = wb.active

search = "4308"
brca_input = "BRCA1"

document.add_heading("allenigmavariants_BICsubmission_2013-07-01")

table = document.add_table(rows=1, cols=4)

table.style = 'Colorful List Accent 1'
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Gene'
hdr_cells[1].text = 'HGSV nucleotide'
hdr_cells[2].text = 'BIC nucleotide'
hdr_cells[3].text = 'Class'


for row in sheet.iter_rows():
    for cell in row:
        if brca_input in str(cell.value) and search in str(cell.offset(column=1).value):
            row_cells = table.add_row().cells
            row_cells[0].text = cell.value
            row_cells[1].text = cell.offset(column=1).value
            row_cells[3].text = cell.offset(column=6).value
            Result = False
            
        elif brca_input in str(cell.value) and search in str(cell.offset(column=3).value):
            row_cells = table.add_row().cells
            row_cells[0].text = cell.value
            row_cells[2].text = cell.offset(column=3).value
            row_cells[3].text = cell.offset(column=6).value
            Result = False


document.save('test.docx')


In [49]:
#3)Reading excel sheet: Published Multifactoral data enigma


wb = load_workbook(folder_location + enigma)

sheet = wb.active


search = "2002"
brca_input = "BRCA1"

document.add_heading("BRCA_Multifac_published data_ENIGMAwebsite 2015-03-27")

table = document.add_table(rows=1, cols=4)

table.style = 'Colorful List Accent 1'
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Gene'
hdr_cells[1].text = 'HGSV nucleotide'
hdr_cells[2].text = 'BIC nucleotide'
hdr_cells[3].text = 'Class'


for row in sheet.iter_rows():
    for cell in row:
        if brca_input in str(cell.value) and search in str(cell.offset(column=1).value):
            row_cells = table.add_row().cells
            row_cells[0].text = cell.value
            row_cells[1].text = cell.offset(column=1).value
            row_cells[3].text = cell.offset(column=6).value
            Result = False
        elif brca_input in str(cell.value) and search in str(cell.offset(column=3).value):
            row_cells = table.add_row().cells
            row_cells[0].text = cell.value
            row_cells[2].text = cell.offset(column=3).value
            row_cells[3].text = cell.offset(column=6).value
            Result = False

document.save('test.docx')


In [59]:
#4) Reading excel sheet: bicbnl 27  fh 2-10-13_nov2013

wb = load_workbook(folder_location + bicbnl)

search = "2685"
brca_input = "BRCA1"

document.add_heading("bicbnl 27  fh 2-10-13_nov2013")

table = document.add_table(rows=1, cols=5)

table.style = 'Colorful List Accent 1'
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Gene'
hdr_cells[1].text = 'c.nom'
hdr_cells[2].text = 'Type'
hdr_cells[3].text = 'opm LOB indeling'
hdr_cells[4].text = 'opmerking'

if brca_input == "BRCA1":
    sheet = wb['totaal BRCA1']

    for row in sheet.iter_rows(max_col = 5, min_col = 5):
        for cell in row:
            if search in str(cell.value):
                row_cells = table.add_row().cells
                row_cells[0].text = brca_input
                row_cells[1].text = cell.value
                row_cells[2].text = str(cell.offset(column=2).value)
                row_cells[3].text = str(cell.offset(column=14).value)
                row_cells[4].text = str(cell.offset(column=15).value)
                Result = False

elif brca_input == "BRCA2":
    sheet = wb['totaal BRCA2']

    for row in sheet.iter_rows(max_col = 5, min_col = 5):
        for cell in row:
            if search in str(cell.value):
                row_cells = table.add_row().cells
                row_cells[0].text = brca_input
                row_cells[1].text = cell.value
                row_cells[2].text = str(cell.offset(column=2).value)
                row_cells[3].text = str(cell.offset(column=14).value)
                row_cells[4].text = str(cell.offset(column=15).value)
                Result = False

document.save('test.docx')


In [15]:
#Setting up selenium driver
#Install the corresponding version of chromedriver on https://sites.google.com/a/chromium.org/chromedriver/downloads and place the file in a map. Store the path to the file in the PATH variable below.
option = webdriver.ChromeOptions()
option.add_argument('headless')
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH, options=option)


In [37]:

#Searching in the LOVD database
option = webdriver.ChromeOptions()
option.add_argument('headless')
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH, options=option)

brca_input = "BRCA1"
if brca_input == "BRCA1":
    driver.get("https://databases.lovd.nl/shared/variants/BRCA1/unique")
    searchbar = driver.find_element_by_name("search_VariantOnTranscript/DNA")
    search = "5095"
    searchbar.send_keys(search)
    searchbar.send_keys(Keys.RETURN)
    
    time.sleep(1)

    document.add_heading("LOVD")
    table_word = document.add_table(rows=1, cols=3)
    table_word.style = 'Colorful List Accent 1'
    hdr_cells = table_word.rows[0].cells
    hdr_cells[0].text = 'Gene'
    hdr_cells[1].text = 'DNA Change (cDNA)'
    hdr_cells[2].text = 'Clinical Classification'
    

    count = 0
    table = driver.find_element_by_xpath("//table[@id='viewlistTable_CustomVL_VOTunique_VOG_BRCA1']")
    for row in table.find_elements_by_xpath("//tr[@class='data']"):
        row_cells = table_word.add_row().cells
        count = 0
        for table_data in row.find_elements_by_xpath(".//td"):
            row_cells[0].text = brca_input
            count = count + 1
            if count == 4:
                row_cells[1].text = table_data.text
            if count == 9:
                row_cells[2].text = table_data.text

    driver.quit()

elif brca_input == "BRCA2":
    driver.get("https://databases.lovd.nl/shared/variants/BRCA2/unique")
    searchbar = driver.find_element_by_name("search_VariantOnTranscript/DNA")
    search = "5095"
    searchbar.send_keys(search)
    searchbar.send_keys(Keys.RETURN)
    
    time.sleep(1)

    document.add_heading("LOVD")
    table_word = document.add_table(rows=1, cols=3)
    table_word.style = 'Colorful List Accent 1'
    hdr_cells = table_word.rows[0].cells
    hdr_cells[0].text = 'Gene'
    hdr_cells[1].text = 'DNA Change (cDNA)'
    hdr_cells[2].text = 'Clinical Classification'
    
    
    count = 0
    table = driver.find_element_by_xpath("//table[@id='viewlistTable_CustomVL_VOTunique_VOG_BRCA2']")
    for row in table.find_elements_by_xpath("//tr[@class='data']"):
        row_cells = table_word.add_row().cells
        count = 0
        for table_data in row.find_elements_by_xpath(".//td"):
            row_cells[0].text = brca_input
            count = count + 1
            if count == 4:
                row_cells[1].text = table_data.text
            if count == 9:
                row_cells[2].text = table_data.text

    driver.quit()
document.save('test.docx') 

In [38]:
# Searching BRCA Exchange database
search = "5073"
brca_input = "BRCA2"

option = webdriver.ChromeOptions()
option.add_argument('headless')
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH, options=option)

document.add_heading("BRCA Exchange")
table_word = document.add_table(rows=1, cols=3)
table_word.style = 'Colorful List Accent 1'
hdr_cells = table_word.rows[0].cells
hdr_cells[0].text = 'Gene'
hdr_cells[1].text = 'GVS Nucleotide'
hdr_cells[2].text = 'Clinical Classification'


driver.get("https://brcaexchange.org/")

searchbar = driver.find_element_by_xpath("//input[@type='text']")
searchbar.send_keys(brca_input + " " +search)
searchbar.send_keys(Keys.RETURN)

time.sleep(5)
table = driver.find_element_by_xpath("//*[@id='data-table-container']/div/table/tbody")

for row in table.find_elements_by_xpath("//tr"):
    count = 0
    row_cells = table_word.add_row().cells
    for table_data in row.find_elements_by_xpath('.//td'):
        count = count + 1
        if count == 1:
            row_cells[0].text = table_data.text
        if count == 2:
            row_cells[1].text = table_data.text
        if count == 6:
            row_cells[2].text = table_data.text
driver.quit()
document.save('test.docx')

In [51]:


search = "5073"
brca_input = "BRCA1"
if brca_input == "BRCA1":

    option = webdriver.ChromeOptions()
    option.add_argument('headless')
    PATH = "C:\Program Files (x86)\chromedriver.exe"
    driver = webdriver.Chrome(PATH, options=option)

    document.add_heading("BRCA Exchange")
    table_word = document.add_table(rows=1, cols=3)
    table_word.style = 'Colorful List Accent 1'
    hdr_cells = table_word.rows[0].cells
    hdr_cells[0].text = 'Gene'
    hdr_cells[1].text = 'Transcript Variant'
    hdr_cells[2].text = 'Functional CLass'


    driver.get("https://sge.gs.washington.edu/BRCA1/#tab-9159-2")

    time.sleep(1)
    tab = driver.find_element_by_xpath("//*[@id='main_page']/div/div/nav/div/ul/li[2]/a")
    tab.click()
    '''
    tab = driver.find_element_by_xpath("//ul[@class='nav navbar-nav']")
    tab.find_element_by_xpath(".//a[@href='#tab-9159-2']").click()
    '''
    time.sleep(5)

    searchbar = driver.find_element_by_xpath("//*[@id='DataTables_Table_0_filter']/label/input")
    searchbar.send_keys(search)
    searchbar.send_keys(Keys.RETURN)
    
    table = driver.find_element_by_xpath("//*[@id='DataTables_Table_0']/tbody")
 
    count = 0
    time.sleep(5)
    hit = False

    for row in table.find_elements_by_xpath("//tr"):
        row_cells = table_word.add_row().cells
        count = 0
        for table_data in row.find_elements_by_xpath((".//td")):
            count = count + 1
            if count == 6 and search in table_data.text:
                row_cells[0].text = brca_input
                hit = True
                row_cells[1].text = table_data.text 
            if count == 10 and hit == True:
                row_cells[2].text = table_data.text
                hit = False
     
driver.quit()
document.save('test2.docx')

exp. chr. pos.hg19 ref alt transcript.variant protein.variant conseq. function.score functional.class RNA.score protein.variant.HGVS
exon3 17 41267771 A C c.106T>G p.S36A Missense 0.067965073 FUNC 0.16032109 p.Ser36Ala
exon16 17 41219690 C T c.5009G>A p.R1670K Missense -1.552697925 LOF -0.650732372 p.Arg1670Lys
exon16 17 41219626 T A c.5073A>T p.T1691= Splice region -1.485894342 LOF -2.71412048 p.Thr1691=
exon16 17 41219626 T C c.5073A>G p.T1691= Splice region -0.38257879 FUNC -2.712178636 p.Thr1691=
exon16 17 41219626 T G c.5073A>C p.T1691= Splice region -1.426685321 LOF -4.282318624 p.Thr1691=
exon21 17 41201191 G T c.5353C>A p.Q1785K Missense -0.283400305 FUNC 0.062507334 p.Gln1785Lys


In [54]:
########Franklin database

search = "BRCA2:c.50del"

option = webdriver.ChromeOptions()
option.add_argument('headless')
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH, options=option)

document.add_heading("Franklin")
table_word = document.add_table(rows=1, cols=2)
table_word.style = 'Colorful List Accent 1'
hdr_cells = table_word.rows[0].cells
hdr_cells[0].text = 'Gene'
hdr_cells[1].text = 'Classification'
row_cells = table_word.add_row().cells

driver.get("https://franklin.genoox.com/clinical-db/home")

time.sleep(10)

ActionChains(driver).send_keys(Keys.ESCAPE).perform()
searchbar = driver.find_element_by_xpath("/html/body/app-root/div/gnx-home-page/div/gnx-search/div[2]/input")
button = driver.find_element_by_xpath("/html/body/app-root/div/gnx-home-page/div/gnx-search/div[2]/gnx-variant-type-toggle/button[2]")

button.click()

searchbar.send_keys(search)
searchbar.send_keys(Keys.RETURN)

time.sleep(10)

driver.find_element_by_xpath("//*[@id='mat-tab-label-2-3']").click()

time.sleep(3)

classification = driver.find_element_by_xpath("//*[@class='indicator-text ng-star-inserted']")

row_cells[0].text = brca_input
row_cells[1].text = classification.text
franklin_class = classification.text

document.save('test2.docx')
driver.quit()


Likely Pathogenic


In [108]:
#Clinvar
brca_input = "BRCA2"
search = "c.5073del"
option = webdriver.ChromeOptions()
option.add_argument('headless')
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH, options = option)

driver.get("https://www.ncbi.nlm.nih.gov/clinvar/")

document.add_heading("Clinvar")
table_word = document.add_table(rows=1, cols=2)
table_word.style = 'Colorful List Accent 1'
hdr_cells = table_word.rows[0].cells
hdr_cells[0].text = 'Classification'
hdr_cells[1].text = 'Review Status'
row_cells = table_word.add_row().cells

time.sleep(1)

searchbar = driver.find_element_by_xpath("//*[@id='term']")
searchbar.send_keys(brca_input + " " + search)
searchbar.send_keys(Keys.RETURN)

table = driver.find_element_by_xpath("//*[@id='tabdocsumtable']/tbody")

driver.find_element_by_xpath("//*[@id='tabdocsumtable']/tbody/tr/td[2]/div/a").click()

Classification = driver.find_element_by_xpath("//*[@id='main_content']/main/div[3]/div/div/dl/dd[1]")

row_cells[0].text = classification.text

time.sleep(3)
star_count = 0
star_list = driver.find_elements_by_xpath("//*[@id='main_content']/main/div[3]/div/div/dl/dd[2]/a/span")
stars = driver.find_elements_by_class_name("fa-star")
for star in stars:
    star_count = star_count + 1

row_cells[1].text = star_count * "*"
'''
for row in table.find_elements_by_xpath(".//tr"):
        row_cells = table_word.add_row().cells
        count = 0
        for table_data in row.find_elements_by_xpath((".//td")):
            count = count + 1
            if count == 2:
                row_cells[0].text = table_data.text
            if count == 6:
                row_cells[1].text = table_data.text
'''
driver.quit()
document.save("test2.docx")



In [122]:
#Varsome login required
'''
brca_input = "BRCA2"
search = "c.5073del"
option = webdriver.ChromeOptions()
option.add_argument('headless')
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

driver.get("https://varsome.com/")

document.add_heading("Varsome")
table_word = document.add_table(rows=1, cols=2)
table_word.style = 'Colorful List Accent 1'
hdr_cells = table_word.rows[0].cells
hdr_cells[0].text = "Gene"
hdr_cells[1].text = 'Classification'
row_cells = table_word.add_row().cells

time.sleep(1)

searchbar = driver.find_element_by_xpath("//*[@id='search']")
searchbar.send_keys(brca_input + " " + search)
searchbar.send_keys(Keys.RETURN)

time.sleep(1)

cookie = driver.find_element_by_xpath("//*[@id='onetrust-accept-btn-handler']")
cookie.click()

button = driver.find_element_by_xpath("//*[@id='proceedBtn']/h4")
button.click()

time.sleep(10)

classification = driver.find_element_by_xpath("//*[@id='acmg_annotation']/div/div[1]/div/div[2]/div/div/div[1]/div/div/span/div[2]/div/div")

row_cells = table_word.add_row().cells
row_cells[0].text = brca_input + search
row_cells[1].text = classification.text

document.save('test2.docx')
driver.quit()
'''

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='acmg_annotation']/div/div[1]/div/div[2]/div/div/div[1]/div/div/span/div[2]/div/div"}
  (Session info: chrome=90.0.4430.93)
